In [1]:
import os

interface_dir = os.environ["DATA"] + "webinterfaces/exp02/"

tasks_dir = os.path.join(interface_dir, "res", "tasks")
results_dir = os.path.join(interface_dir, "results")
protocols_dir = os.path.join(interface_dir, "protocols")
prolific_matching_dir = os.path.join(interface_dir, "prolific")

protocol_paths_d = {
    "H": os.path.join(protocols_dir, "H_0.json"),
    "H+AI": os.path.join(protocols_dir, "AI_0.json"),
    "H+AI+CF": os.path.join(protocols_dir, "XAI_CF_0.json"),
    "H+AI+SHAP": os.path.join(protocols_dir, "XAI_SHAP_0.json"),
    "H+AI+LLM": os.path.join(protocols_dir, "XAI_LLM_0.json"),
    "H+AI+GRADCAM": os.path.join(protocols_dir, "XAI_GRADCAM_0.json"),
}

COMPREHENSION_THRESHOLD = 0.8

COMPREHENSION_TASKS = ["xeasy1_find_pattern_rot"]
TRAINING_TASKS = ["med3_find_pattern_rot"]
EASY_TASKS = ["easy1_find_pattern_rot", "easy3_find_pattern_rot"]
DIFFICULT_TASKS = ["hard1_find_pattern_rot", "hard3_find_pattern_rot"]

MILD_PRESSURE_TASKS = ["easy1_find_pattern_rot", "hard1_find_pattern_rot"]
STRONG_PRESSURE_TASKS = ["easy3_find_pattern_rot", "hard3_find_pattern_rot"]

TASK_PROTOCOL_KEYS = {
    "easy1_find_pattern_rot": "mainexp_easy_mild_patrot_task",
    "easy3_find_pattern_rot": "mainexp_easy_strong_patrot_task",

    "hard1_find_pattern_rot": "mainexp_hard_mild_patrot_task",
    "hard3_find_pattern_rot": "mainexp_hard_strong_patrot_task",

    "xeasy1_find_pattern_rot": "intro_comprehension_task",
    "med3_find_pattern_rot": "intro_training_1_task"
}

full_bonus_pay = 53*0.04


In [2]:
import json

def data_matching(protocols_paths_d, prolific_matching_files):

    results_filenames_d = {k: [] for k in protocol_paths_d.keys()}

    for prolific_matching_file in prolific_matching_files:

        with open(prolific_matching_file) as json_data:
            d = json.load(json_data)

            for prolific_id, prot_dict in d.items():
                condition_split = prot_dict["protocol"].split("_")
                filename = prolific_id + ".json"

                if condition_split[0] == "H":
                    results_filenames_d["H"].append(filename)
                elif condition_split[0] == "AI":
                    results_filenames_d["H+AI"].append(filename)
                elif condition_split[0] == "XAI" and condition_split[1] == "SHAP":
                    results_filenames_d["H+AI+SHAP"].append(filename)
                elif condition_split[0] == "XAI" and condition_split[1] == "CF":
                    results_filenames_d["H+AI+CF"].append(filename)
                elif condition_split[0] == "XAI" and condition_split[1] == "LLM":
                    results_filenames_d["H+AI+LLM"].append(filename)
                elif condition_split[0] == "XAI" and condition_split[1] == "GRADCAM":
                    results_filenames_d["H+AI+GRADCAM"].append(filename)

    return results_filenames_d

In [3]:
# results_filenames_d = data_matching(protocol_paths_d, [os.path.join(prolific_matching_dir, "prolific.json")])

In [4]:
results_filenames_d = data_matching(protocol_paths_d, [os.path.join(prolific_matching_dir, "prolific_28-1.json")])


In [5]:
results_filenames_d

{'H': ['6710a58eeff954cb2243950e.json',
  '62e023ed4b0fdb8c95a9e47f.json',
  '6124131685a81a6996d97649.json',
  '665b5d015bd1fbd411f8b77a.json',
  '6128279b17af1258eba39ada.json',
  '6978cf2c491bc6211008dd30.json',
  '66656db67864801db6a0e2c6.json',
  '6664c7aa5c1c7846d3618f74.json',
  '63ee6ca92186ac65c4437d62.json',
  '5f620fdcc464341e830b3866.json',
  '6659cb22b8e889fe11ae56d2.json',
  '5bc73b9f821054000181a853.json',
  '69657262923a8424b8926875.json',
  '5b472672d4c93b00019bbc49.json',
  '55ac3198fdf99b6de3b68b23.json',
  '59cb95053306be000195be2e.json',
  '66774233add67a4a8e71398e.json',
  '5a70b6b882968f0001a6b12f.json'],
 'H+AI': ['5a2ef9bbfd28e9000102ee68.json',
  '6978c567f25eda92929111ae.json',
  '5d68c8aa40524c00189e8ac2.json',
  '63d13ca3a2e69ec8e19569f2.json',
  '62347ee217c402f4acae35d0.json',
  '62b05691722d0db3c9d69acb.json',
  '6658bac05e1a41dd4554301a.json'],
 'H+AI+CF': ['60fde2f0d7936b5e07c3955b.json',
  '6978de0358edaccb1d2b58b6.json',
  '5c7018a5e165d10001e8aed2.j

In [6]:
for k, v in results_filenames_d.items():
    print(f"{k}: {len(v)}")

H: 18
H+AI: 7
H+AI+CF: 13
H+AI+SHAP: 11
H+AI+LLM: 11
H+AI+GRADCAM: 6


In [7]:
import sys

sys.path.append("/home/jleguy/Documents/postdoc/git_repos/WebXAII/")

In [8]:
import json
import csv
import numpy as np


def load_json(path):
    with open(path) as json_file:
        return json.load(json_file)


def load_task_csv_file(path):
    y_true, y_pred = [], []
    with open(path) as csv_data:
        reader = csv.DictReader(csv_data)
        for row in reader:
            y_true.append(int(row["target"]))
            y_pred.append(int(row["pred"]))

    return np.array(y_true), np.array(y_pred)


In [9]:
from pywebxaii.resretrieval import extract_p_task_results


def compute_scores_by_key(results_dir, results_filenames_d, protocol_paths_d, tasks_dir, tasks_names, task_protocol_keys):
    output_res_scores_d = {}

    # Iterating over all groups
    for group_key, filenames_list in results_filenames_d.items():

        output_res_scores_d[group_key] = {}

        # Iterating on all results files for the current group
        for filename in filenames_list:
            print(filename)
            prolific_id = filename.split(".")[0]
            curr_res_path = os.path.join(results_dir, filename)
            curr_res_d = load_json(curr_res_path)
            if not curr_res_d["is_completed"]:
                continue

            nb_questions = 0
            nb_quest_wrong_predictions = 0
            nb_quest_right_predictions = 0
            nb_correct = 0
            early_break = False

            # Iterating over all tasks
            for task_idx, task_name in enumerate(tasks_names):

                task_true, ai_pred = load_task_csv_file(os.path.join(tasks_dir, task_name + "_content.csv"))

                answers_idx_vect, answers_text_vect, _, _, _ = \
                    extract_p_task_results(curr_res_d,
                                           task_protocol_keys[tasks_names[task_idx]],
                                           protocol_d=load_json(protocol_paths_d[group_key]))

                nb_questions += len(answers_idx_vect)
                nb_quest_wrong_predictions += np.sum(task_true != ai_pred)
                nb_quest_right_predictions += np.sum(task_true == ai_pred)
                try:
                    nb_correct += np.sum(answers_idx_vect == np.logical_not(task_true))
                    #
                    # if np.isnan(np.sum(answers_idx_vect)):
                    #     print(f"answers {answers_idx_vect}")
                    #     print(f"true {np.logical_not(task_true)}")
                    #     print(f"ai pred {np.logical_not(ai_pred)}")
                    #     print(f"correct extracted {np.sum(answers_idx_vect == np.logical_not(task_true))}")
                    #     print(f"reliance extracted {np.sum(answers_idx_vect == np.logical_not(ai_pred))}")

                # Happens if the results file is not complete
                except ValueError:
                    print("ValueError exception")
                    output_res_scores_d[group_key][prolific_id] = None

                    early_break = True
                    break

            if not early_break:
                output_res_scores_d[group_key][prolific_id] = nb_correct / nb_questions

    return output_res_scores_d


In [10]:
def filter_not_completed(results_filenames_d):
    filtered_results_filenames_d = {}

    # Iterating over all groups
    for group_key, filenames_list in results_filenames_d.items():

        filtered_results_filenames_d[group_key] = []

        # Iterating on all results files for the current group
        for filename in filenames_list:
            curr_res_path = os.path.join(results_dir, filename)
            curr_res_d = load_json(curr_res_path)
            if not curr_res_d["is_completed"]:
                continue
            filtered_results_filenames_d[group_key].append(filename)
    return filtered_results_filenames_d

In [11]:
results_filenames_d = filter_not_completed(results_filenames_d)

In [12]:
all_scores = compute_scores_by_key(results_dir, results_filenames_d, protocol_paths_d,
                                                                               tasks_dir,
                                                                               EASY_TASKS + DIFFICULT_TASKS + COMPREHENSION_TASKS, TASK_PROTOCOL_KEYS)

6710a58eeff954cb2243950e.json


ValueError: too many values to unpack (expected 5)

In [64]:
all_scores

{'H': {'6710a58eeff954cb2243950e': 0.6415094339622641,
  '6124131685a81a6996d97649': 0.7547169811320755,
  '665b5d015bd1fbd411f8b77a': 0.8301886792452831,
  '6978cf2c491bc6211008dd30': 0.6981132075471698,
  '66656db67864801db6a0e2c6': 0.660377358490566,
  '6664c7aa5c1c7846d3618f74': 0.49056603773584906,
  '63ee6ca92186ac65c4437d62': 0.6792452830188679,
  '5f620fdcc464341e830b3866': 0.8679245283018868,
  '6659cb22b8e889fe11ae56d2': 0.7924528301886793,
  '5bc73b9f821054000181a853': 0.6981132075471698,
  '69657262923a8424b8926875': 0.7358490566037735,
  '5b472672d4c93b00019bbc49': 0.7924528301886793,
  '55ac3198fdf99b6de3b68b23': 0.7169811320754716,
  '66774233add67a4a8e71398e': 0.7924528301886793},
 'H+AI': {'5a2ef9bbfd28e9000102ee68': 0.7358490566037735,
  '6978c567f25eda92929111ae': 0.8867924528301887,
  '5d68c8aa40524c00189e8ac2': 0.9245283018867925,
  '63d13ca3a2e69ec8e19569f2': 0.8867924528301887,
  '62347ee217c402f4acae35d0': 0.8113207547169812,
  '62b05691722d0db3c9d69acb': 0.8113

In [65]:
def gen_bonus_files(scores, bonus_reward_location, accepted_location, full_bonus_pay):
    csv_data_reward = []
    csv_data_accepted = []
    payment_sum = 0
    participants = 0

    for group, d in scores.items():
        for prolific_id, score in d.items():
            payment = score*full_bonus_pay
            csv_data_reward.append([prolific_id, f"{payment:.2f}"])
            csv_data_accepted.append([prolific_id])
            payment_sum += payment
            participants += 1

    with open(bonus_reward_location, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(csv_data_reward)

    with open(accepted_location, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(csv_data_accepted)

    print(f"Total payment for {participants} participants is {payment_sum}, corresponding to a mean value of {payment_sum/participants}")


In [66]:
gen_bonus_files(all_scores, os.path.join(results_dir, "rewards_28-1.csv"), os.path.join(results_dir, "accepted_28-1.csv"),
                full_bonus_pay)

Total payment for 56 participants is 94.95999999999998, corresponding to a mean value of 1.6957142857142853
